In [37]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

#from ModelClassifier import PAClassifier,SGDSVMClassifier,XGBoostClassifier

In [28]:
FeaturesSelected=np.load('./mostrecentinput/SDGSVM_Oct172015231303_AUC_0.748204837403.p')
X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
X=np.hstack((X,X1,X2,X3))
y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))
xtrain=X[:,FeaturesSelected]
del X,X1,X2,X3

X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X1_test= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X2_test=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X3_test=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
X_test=np.hstack((X_test,X1_test,X2_test,X3_test))

xtest=X_test[:,FeaturesSelected]
del X_test,X1_test,X2_test,X3_test

TestID=pd.read_pickle("./mostrecentinput/TestID.p")

In [127]:
KFoldNumber=3
niter=400
    
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_train=[]
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
indexes_cv=[]
cv_values1=np.zeros(KFoldNumber)

i=0
model = linear_model.SGDClassifier()
kfoldnum=0

train_predicted=np.zeros(y.shape)

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    preds_cv=0
    preds_test=0
    preds_train=0
    ncount=0
    cv_index=np.zeros(y.shape)
    for i in range(niter):
        clf = model.fit(xtrain[train,:], y[train])
        predcv0=clf.predict(xtrain[cv,:])
        predtest0=clf.predict(xtest)
        predtrain0=clf.predict(xtrain)
        preds_cv=preds_cv+predcv0
        preds_test=preds_test+predtest0
        preds_train=preds_train+predtrain0
        ncount+=1
        if np.mod(i,round(niter/10))==0:print(i,end=' ')
    print(' ')
    preds_cv=preds_cv/float(ncount)
    preds_test=preds_test/float(ncount)
    preds_train=preds_train/float(ncount)
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv) 
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_test
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_train
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    cv_values1[kfoldnum-1]=float(AUC1)
    print(cv_values1)
    i+=1
    
    
str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename='FinalModelPrediction_SVM_'  +  str1 + '_niter400_AUC'+ str(cv_values1.max())+ '.p'
pickle.dump((ypreds_test_wl,ypreds_train_wl,ypreds_cvindex_wl,cv_values1), 
            open(filename,'wb'), protocol =2)  
print('Model prediction done!')

0 40 80 120 160 200 240 280 320 360  
[ 0.7475594  0.         0.       ]
0 40 80 120 160 200 240 280 320 360  
[ 0.7475594   0.75659574  0.        ]
0 40 80 120 160 200 240 280 320 360  
[ 0.7475594   0.75659574  0.75444948]
